# Задание №1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

Наименование вакансии. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам). Ссылку на саму вакансию. Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
import requests
from bs4 import BeautifulSoup as bs

from time import sleep
import json

import pandas as pd

# Вариант 1: через API hh

In [2]:
def hh_find_job_from_API(vacancy, page_number):
    url = 'https://api.hh.ru/vacancies'
    sleep(3)
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 OPR/95.0.4635.25',
    'content-encoding': 'gzip',
    'content-type': 'text/html'
        }
    params = {
        'text': f'NAME:{vacancy}', # Искомая вакансия
        'page': f'{page_number}', # Индекс страницы поиска на HH
        'per_page': 20 # Кол-во вакансий на 1 странице
    }
    
    response = requests.get('https://api.hh.ru/vacancies', params)
    data = response.content.decode() # Декодируем ответ, чтобы Кириллица отображалась корректно
    
    jsObj = json.loads(data)
    result = {}
    
    for i in jsObj['items']:
        name = i['name']
        result[name] = {
            'Salary': i['salary'],
            'Location': i['area']['name'],
            'Employer': i['employer']['name'],
            'Link': i['alternate_url']
        }

    with open('hh_find_job_from_API.json', 'w') as f:
        json.dump(result, f)
        
    hh_jobs = pd.read_json('hh_find_job_from_API.json').T
    return hh_jobs
    


In [9]:
hh_find_job_from_API('python', '0')

,Salary,Location,Employer,Link
Разработчик Python,"{'from': 200000, 'to': 250000, 'currency': 'RU...",Санкт-Петербург,Нетопия,https://hh.ru/vacancy/77091466
Ведущий Python разработчик (удаленка),"{'from': None, 'to': 8000, 'currency': 'USD', ...",Москва,Apostro,https://hh.ru/vacancy/76832091
Backend программист (Python),"{'from': 120000, 'to': None, 'currency': 'RUR'...",Москва,MANGO FZCO,https://hh.ru/vacancy/77147428
"Стажер-разработчик Python (компьютерное зрение), удалённо","{'from': 15000, 'to': None, 'currency': 'RUR',...",Санкт-Петербург,Nti.team,https://hh.ru/vacancy/77172833
Ведущий Python-developer,"{'from': 220000, 'to': 340000, 'currency': 'RU...",Екатеринбург,Тензор,https://hh.ru/vacancy/77013935
Программист Python Junior,None,Екатеринбург,BRANDPOL,https://hh.ru/vacancy/77146976
Разработчик бэкэнд (Python),"{'from': 150000, 'to': 180000, 'currency': 'RU...",Москва,АНО Инфокультура,https://hh.ru/vacancy/76712573
Python developer,"{'from': 120000, 'to': 240000, 'currency': 'RU...",Санкт-Петербург,Subtotal,https://hh.ru/vacancy/77107980
Программист Python - FastAPI,"{'from': 180000, 'to': None, 'currency': 'RUR'...",Ростов-на-Дону,Digital агентство Piarme,https://hh.ru/vacancy/77096692
"Python-разработчик (Django, DRF)","{'from': 60000, 'to': 220000, 'currency': 'RUR...",Москва,Фабрика Решений,https://hh.ru/vacancy/70705466


# Вариант 2: с использованием BeautifulSoup

In [39]:
def hh_find_job(vacancy, page_number):
    url = f'https://hh.ru/search/vacancy?text={str(vacancy)}&salary=&ored_clusters=true&enable_snippets=true&page={str(page_number)}'
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 OPR/95.0.4635.25',
    'content-encoding': 'gzip',
    'content-type': 'text/html'
        }
    
    sleep(3)
    response = requests.get(url, headers=headers)
    soup = bs(response.content, 'html.parser')
    
    root = soup.find_all('div', attrs={'class': 'vacancy-serp-item-body__main-info'})
    
    result = {}
    for i in range(len(root)):
        try:
            name = root[i].find('h3').find('span').find('a', attrs={'class': 'serp-item__title'}).text
            salary = (root[i].find('span', attrs={'class': 'bloko-header-section-3'}).get_text()).replace('\u202f', '').replace('\xa0', '')
            link = root[i].find('h3').find('span').find('a', attrs={'class': 'serp-item__title'})['href']
            employer = (root[i].find_next('a', attrs={'class': 'bloko-link bloko-link_kind-tertiary'}).text).replace('\xa0', ' ')
            location = (root[i].find_next('span', attrs={'class': 'metro-station'}).parent.text).replace('\xa0', ' ')
        except AttributeError:
            salary = 'З/п не указана'
            
        result[name] = {
            'Salary': salary,
            'Employer': employer,
            'Location': location,
            'Link': link
        }
    
    with open('hh_find_job_from_API.json', 'w') as f:
        json.dump(result, f)
        
    hh_jobs = pd.read_json('hh_find_job_from_API.json').T
    return hh_jobs

In [40]:
hh_find_job('', '0')

,Salary,Employer,Location,Link
Водитель,до 200000 руб.,ООО М.М.А.,"Москва, Комсомольская и еще 3",https://hh.ru/vacancy/77165803?from=vacancy_se...
Оператор ПК (удалённая работа),50000 – 80000 руб.,ProZdorovie,"Москва, Славянский бульвар",https://hh.ru/vacancy/76918201?from=vacancy_se...
Продавец-консультант в магазины lady & gentleman CITY,50000 – 65000 руб.,Леди и Джентльмен,"Москва, Славянский бульвар",https://hh.ru/vacancy/38708211?from=vacancy_se...
Водитель с функциями помощника,80000 – 100000 руб.,Сорокина Марина Сергеевна,"Москва, Славянский бульвар",https://hh.ru/vacancy/76908164?from=vacancy_se...
Менеджер по продажам в отделение Банка,от 65000 руб.,ПАО Банк Санкт-Петербург,"Москва, Славянский бульвар",https://hh.ru/vacancy/42909942?from=vacancy_se...
Менеджер по работе с маркетплейсами,от 60000 руб.,ООО Кофе Рум,"Москва, Славянский бульвар",https://hh.ru/vacancy/77020074?from=vacancy_se...
"Вице-президент по развитию новых продуктов и рынков (производственно-торговая компания), Казахстан",1500000 – 5000000 KZT,ТОО TOP Personal,"Москва, Славянский бульвар",https://hh.ru/vacancy/77130908?from=vacancy_se...
Водитель категория В,от 1700 бел.руб.,ООО ВестАгроТранс,"Москва, Славянский бульвар",https://hh.ru/vacancy/77132662?from=vacancy_se...
Секретарь на ресепшен / Администратор,от 70000 руб.,SKOLKOVO GOLF CLUB,"Москва, Славянский бульвар",https://hh.ru/vacancy/76966614?from=vacancy_se...
Продавец консультант в Вейпшопе,130000 – 250000 KZT,ИП ARYZKHAN COMPANY LIMITED,"Москва, Комсомольская и еще 3",https://hh.ru/vacancy/77167268?from=vacancy_se...
